In [184]:
import os
import pandas as pd
from tqdm.notebook import tqdm
def is_folder_empty(folder_path):
    """
    检查指定的文件夹是否为空文件夹。

    参数:
    folder_path (str): 要检查的文件夹的路径

    返回:
    bool: 如果文件夹为空，返回True；否则返回False。
    """
    if not os.path.exists(folder_path):
        raise ValueError(f"指定的路径 {folder_path} 不存在，请检查输入的文件夹路径是否正确。")
    if not os.path.isdir(folder_path):
        raise ValueError(f"{folder_path} 不是一个文件夹，请确保输入的是文件夹路径。")

    # 获取文件夹下的所有文件和子文件夹列表
    contents = os.listdir(folder_path)
    return len(contents) == 0

## test embed on Relation Extraction Task(RE)

In [14]:
RE_train = pd.read_json('../train/RE/RE-train.json')

In [35]:
def RE_Embed_Text(df): ## 返回3列分别是text_1,text_2,label
    SimTab = df
    context_list = []
    target_list = []
    label_list = []
    for i in range(len(SimTab)):
        text = SimTab.iloc[i,0]
        label = list(eval(SimTab.iloc[i,2]).values())[0]
        label_list.append(label) ## add label
        context = text.split('\n\nTable1:')[1].split('\n\nRelation Option: \n\n')[0].replace('<table_title>','<table_title> ').replace('<header>',' <header> ').replace('|',' | ')
        context_list.append(str(context)) ## add context
        target = text.split('<header>')[1].split('\n\nRelation Option: \n\n')[0]
        target_list.append(str(target.replace('|',' | ')))
    return context_list,target_list,label_list
def CTA_Embed_Text(df): ## 返回3列分别是text_1,text_2,label
    SimTab = df
    context_list = []
    target_list = []
    label_list = []
    for i in range(len(SimTab)):
        text = SimTab.iloc[i,0]
        label = list(eval(SimTab.iloc[i,2]).values())[0]
        label_list.append(label) ## add label
        context = text.split('Table 1:\n\n')[1].split('\n\nReference tables:')[0].split('\n')
        context_list.append(str(context)) ## add context
        
        target = text.split('\n\nColumn: ')[1].split('\n\nOptions:')[0]
        related_context = []
        for candidate in context:
            try:
                dict_output = eval(candidate)
                related_context.append(dict_output[target])
            except:
                print('fail')
                print(dict_output.keys(),target)
        target_list.append(str({target:related_context}))
    return context_list,target_list,label_list

In [30]:
c,t,l = RE_Embed_Text(RE_train)

In [36]:
CTA_WebTable = pd.read_json('../train/CTA/WebTable/WebTable-train.json')

In [37]:
# c,t,l = RE_Embed_Text(RE_train)
c,t,l = CTA_Embed_Text(CTA_WebTable)

{'init_select': 143.5319640636444, 'init-train': 143.53495621681213, 'batch-division': 165.7140564918518, 'gradient-calculation': 263.05509662628174, 'IF-Score': 303.7691614627838, 'Final Selection': 304.0284495353699} CTA-WebTable 14K data

In [43]:
ER_train = pd.read_json('../../MELD/dataset/ER/wdc_all_train_output.json')

In [76]:
def ER_Embed_Text(df): ## 返回3列分别是text_1,text_2,label
    SimTab = df
    context_list = []
    target_list = []
    label_list = []
    for i in range(len(SimTab)):
        text = SimTab.iloc[i,0]
        label = list(eval(SimTab.iloc[i,2]).values())[0]
        label_list.append(label) ## add label
        context = text.split('Output format example:{"Output": ""}\n\nEntity 1:')[1].split('\n\nEntity 2:')[0].replace('\\n', '').replace('\n', '').replace('\\', '')
        context_list.append(context) ## add context
        
        target = text.split('\n\nEntity 2:')[1].split('\n\nTake these examples as reference:')[0].replace('\\n', '').replace('\n', '').replace('\\', '')
        target_list.append(target)
    return context_list,target_list,label_list

In [165]:
# # print(ER_train.iloc[0,0])
# train = pd.read_json('../../MELD/dataset/ER/semi-text-w-train-MoE.json')
# # print(ER_train.iloc[0,0])
# test = pd.read_json('../../MELD/dataset/ER/semi-text-w-test-MoE.json')

# print(ER_train.iloc[0,0])
train = pd.read_json('../../MELD/dataset/ER/walmart_amazon_train_output.json')
# print(ER_train.iloc[0,0])
test = pd.read_json('../../MELD/dataset/ER/walmart_amazon_test_output.json')

In [168]:
# test.columns = train.columns
train.columns = ['instruction','input','output']
test.columns = ['instruction','input','output']

In [169]:
for column in ['instruction','input','output']:
    train[column] = train[column].str.replace('dismatch','mismatch')
    test[column] = test[column].str.replace('dismatch','mismatch')

In [170]:
# train['instruction'] = train['instruction'].str.split('\n\nTake these examples as reference:')[0]
# test['instruction'] = test['instruction'].str.split('\n\nTake these examples as reference:')[0]
def cut_length(row):
    instruction = row['instruction']
    instruction = instruction.split('\n\nTake these examples as reference:')[0]
    return instruction
train['instruction'] = train.apply(cut_length,axis=1)
test['instruction'] = test.apply(cut_length,axis=1)

In [171]:
len(train)

7103

In [172]:
import json
json.dump(train.to_dict(orient='records'), open('../train/ER/walmart-amazon/walmart-amazon-train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(test.to_dict(orient='records'), open('../train/ER/walmart-amazon/walmart-amazon-test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

{'init_select': 61.000027894973755, 'init-train': 171.5253722667694, 'batch-division': 182.59329438209534, 'gradient-calculation': 228.29285073280334, 'IF-Score': 261.42748260498047, 'Final Selection': 261.6604371070862, 'Selection fine-tune': 509.3796503543854, 'Selection Inference': 590.3308668136597} wdc-time-4K 0.8678544914625093
semi-text-w 0.7230
semi-text-c F1:0.7885597548518897
{'init_select': 56.48771262168884, 'init-train': 165.2728819847107, 'batch-division': 171.84898138046265, 'gradient-calculation': 210.02755188941956, 'IF-Score': 234.4348783493042, 'Final Selection': 234.57274293899536, 'Selection fine-tune': 491.0318133831024, 'Selection Inference': 552.2762784957886} Abt-Buy 0.8957871396895787

{'init_select': 36.64403533935547, 'init-train': 117.11347103118896, 'batch-division': 122.72263956069946, 'gradient-calculation': 150.83489441871643, 'IF-Score': 172.2080376148224, 'Final Selection': 172.37958312034607, 'Selection fine-tune': 347.78654956817627, 'Selection Inference': 408.50533270835876} amazon-google F1:0.782435129740519

{'init_select': 42.48365902900696, 'init-train': 129.5051372051239, 'batch-division': 135.72513437271118, 'gradient-calculation': 169.5727024078369, 'IF-Score': 193.17675495147705, 'Final Selection': 193.3632688522339, 'Selection fine-tune': 507.74013566970825, 'Selection Inference': 569.8979163169861} F1:0.8017817371937639 walmart-amazon

In [157]:
c,t,l = ER_Embed_Text(train)

## Data Imputation Task training data

In [223]:
train = pd.read_json('../../MELD/dataset/DC/beer-train-MoE.json')
# print(ER_train.iloc[0,0])
test = pd.read_json('../../MELD/dataset/DC/beer-test.json')

In [224]:
def DC_Embed_Text(df): ## 返回3列分别是text_1,text_2,label
    SimTab = df
    context_list = []
    target_list = []
    label_list = []
    for i in tqdm(range(len(SimTab))):
        text = SimTab.iloc[i,0]
        label = SimTab.iloc[i,2]
        label_list.append(label) ## add label
        if text.__contains__('You are an expert in Cleaning Hospital Dataset.'):
            context = text.split('Entity 1:\n\n')[1].split('\n\nTake these rows as reference:')[0]
            context_list.append(context) ## add context
            
            context_dict = eval(context)
            
            target = text.split('Output Format Example:\n\n')[1].split('\n\nEntity 1:')[0]
            
            target_attr = list(eval(target).keys())[0]
            
            related_context = context_dict[target_attr]
            target_list.append(str({target_attr:related_context}))
        elif text.__contains__('You are an expert in Cleaning Beers Dataset.'):
            context = text.split('Entity 1:\n\n')[1].split('\n\nThe input')[0]
            context_list.append(context) ## add context
            
            context_dict = eval(context)
            
            target = text.split('Output Format Example:\n\n')[1].split('\n\nEntity 1:')[0]
            
            target_attr = list(eval(target).keys())[0]
            
            related_context = context_dict[target_attr]
            target_list.append(str({target_attr:related_context}))
        elif text.__contains__('You are an expert in Cleaning Rayyan Dataset.'):
            context = text.split('Entity 1:\n\n')[1].split('\n\nThe input')[0]
            context_list.append(context) ## add context
            
            context_dict = eval(context)
            
            target = text.split('Output Format Example:\n\n')[1].split('\n\nEntity 1:')[0]
            
            target_attr = list(eval(target).keys())[0]
            
            related_context = context_dict[target_attr]
            target_list.append(str({target_attr:related_context}))
    return context_list,target_list,label_list

In [225]:
c,t,l = DC_Embed_Text(train)

  0%|          | 0/3767 [00:00<?, ?it/s]

In [231]:
import json
json.dump(train.to_dict(orient='records'), open('../train/DC/beer/beer-train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(test.to_dict(orient='records'), open('../train/DC/beer/beer-test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

### Data Imputation Task

In [257]:
train = pd.read_json('../../MELD/dataset/DI/amazon_train_output_wide.json')
# print(ER_train.iloc[0,0])
test = pd.read_json('../../MELD/dataset/DI/amazon_test_output_wide.json')

In [308]:
train = pd.read_json('../../MELD/dataset/AVE/oa_mine_train_small.json')
# print(ER_train.iloc[0,0])
test = pd.read_json('../../MELD/dataset/AVE/oa_mine_test_small.json')

In [314]:
def AVE_Embed_Text(df): ## 返回3列分别是text_1,text_2,label
    SimTab = df
    context_list = []
    target_list = []
    label_list = []
    for i in tqdm(range(len(SimTab))):
        text = SimTab.iloc[i,0]
        label = SimTab.iloc[i,2]
        label_list.append(label) ## add label
        context = text.split('product title:\n\n')[1].split('\n\nTake these rows as examples:')[0]
        context_list.append(context) ## add context
        
        # context_dict = eval(context)
        
        target = text.split('Output Format Example:\n\n')[1].split('\n\nproduct title:')[0]
        
        # target_attr = list(eval(target).keys())[0]
        
        # related_context = context_dict[target_attr]
        target_list.append(target)

    return context_list,target_list,label_list
c,t,l = AVE_Embed_Text(train)

  0%|          | 0/1443 [00:00<?, ?it/s]

In [301]:
# train = pd.read_json('../../MELD/dataset/SM/CMS_train_few_output.json')
# # # print(ER_train.iloc[0,0])
# test = pd.read_json('../../MELD/dataset/SM/CMS_test_few_output.json')
train = pd.read_json('../../MELD/dataset/SM/synthea_train_few_output.json')
# # print(ER_train.iloc[0,0])
test = pd.read_json('../../MELD/dataset/SM/synthea_test_few_output.json')

In [269]:
def DI_Embed_Text(df): ## 返回3列分别是text_1,text_2,label
    SimTab = df
    context_list = []
    target_list = []
    label_list = []
    for i in tqdm(range(len(SimTab))):
        text = SimTab.iloc[i,0]
        label = SimTab.iloc[i,2]
        label_list.append(label) ## add label
        context = text.split('\n\nEntity 1:')[1].split('\n\nTake these examples as reference:')[0]
        context_list.append(context) ## add context
        
        # context_dict = eval(context)
        
        target = text.split('\n\nOutput format example:')[1].split('\n\nEntity 1:')[0]
        
        # target_attr = list(eval(target).keys())[0]
        
        # related_context = context_dict[target_attr]
        target_list.append(target)

    return context_list,target_list,label_list

In [302]:
def SM_Embed_Text(df): ## 返回3列分别是text_1,text_2,label
    SimTab = df
    context_list = []
    target_list = []
    label_list = []
    for i in range(len(SimTab)):
        text = SimTab.iloc[i,0]
        label = list(eval(SimTab.iloc[i,2]).values())[0]
        label_list.append(label) ## add label
        context = text.split('Output format example:{"Output": ""}\n\nColumn 1:')[1].split('\n\nColumn 2:')[0].replace('\\n', '').replace('\n', '').replace('\\', '')
        context_list.append(context) ## add context
        
        target = text.split('\n\nColumn 2:')[1].split('\n\nTake these examples as reference:')[0].replace('\\n', '').replace('\n', '').replace('\\', '')
        target_list.append(target)
    return context_list,target_list,label_list

In [303]:
# print(train.iloc[42,0])

def cut_length(row):
    instruction = row['instruction']
    instruction = instruction.split('\n\nTake these examples as reference:')[0]
    return instruction
for column in ['instruction','input','output']:
    train[column] = train[column].str.replace('dismatch','mismatch')
    test[column] = test[column].str.replace('dismatch','mismatch')
# train['instruction'] = train.apply(cut_length,axis=1)
# test['instruction'] = test.apply(cut_length,axis=1)
c,t,l = SM_Embed_Text(train)

In [305]:
import json
json.dump(train.to_dict(orient='records'), open('../train/SM/synthea/synthea-train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(test.to_dict(orient='records'), open('../train/SM/synthea/synthea-test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [316]:
import json
json.dump(train.to_dict(orient='records'), open('../train/AVE/oa_mine/oa_mine-train.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
json.dump(test.to_dict(orient='records'), open('../train/AVE/oa_mine/oa_mine-test.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=4)